In [ ]:
from sklearn.linear_model import Lasso
from matplotlib import pyplot as plt
import numpy as np
import cPickle
import pandas as pd
import seaborn as sns
import scipy.io
from __future__ import division

In [ ]:
#%matplotlib inline
#%matplotlib notebook  #uncomment for interactive plotting

#### Load data

In [ ]:
# Load Measurement matrix from file
A = np.loadtxt(open("../data/MeasurementMatrix27by88", 'rb'), delimiter=",", skiprows=0)

In [ ]:
MeanSpeedReduction = np.array([[ 61.996],[ 0.],[ 0.],[ 0.],[ 0.],[ 77.822],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],
    [ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 67.034],[ 0.],[ 66.805],[ 0.],[ 0.],[ 0.],[ 78.645]])

In [ ]:
MeanSpeedReductionError = np.array([[ 5.9132],[ 0.],[ 0.],[ 0.],[ 0.],[ 4.7142],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],
    [ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 0.],[ 15.914],[ 0.],[ 12.438],[ 0.],[ 0.],[ 0.],[ 4.0911]])

In [ ]:
# Phenotype Vector
ymean = MeanSpeedReduction
ystd  = MeanSpeedReductionError

In [ ]:
Neurons = ['ADA','AIA','AIB','AIM','AIN','AIY','AIZ','ALA','AUA','AVA','AVB','AVD','AVE','AVG','AVH','AVJ','AVK','AVL',
 'BDU','CAN','DVA','DVB','DVC','I2','I5','I6','IL1','LUA','PLN','PVC','PVN','PVQ','PVR','PVT','PVW','RIA','RIB','RIC',
 'RID','RIF','RIG','RIP','RIV','RMD','RMF','RMG','RMH','SAA','SAB','SDQ','SIA','SIB','SMB','SMD','URB','URX','ADE',
 'ADF','ADL','ALN','ASE','ASG','ASH','ASI','ASJ','ASK','AWA','AWC','BAG','CEP','FLP','IL2','OLL','PHA','PHB','PHC',
 'PQR','PVD','DD1','HSN','M2','M4','MC','NSM','PDA','RIM','RME','URA']

#### Inference - Sparse solutions

In [ ]:
# Continuous phenotype vector - bootstraping with 10000 solution
# may take few minutes
LX10k=np.zeros((50,10000,88))
L1norm10k=np.zeros((50,10000))
L2norm10k=np.zeros((50,10000))
y10kStored = np.zeros((50,10000))

As=np.logspace(-2,1.5,num=50) #Sparsity parameters
count=0
for a in As:
    ytemp=np.zeros((27,1));
    X=np.zeros((10000,88));
    for i in range(0,10000):
        ytemp = ymean + ystd * np.random.randn(27,1)
        clf = Lasso(alpha=a,max_iter=5000) #Lasso model
        clf.fit(A, ytemp)                  #Fit Lasso model using our measurement matrisx and phenotype vector
        X[i,:] = clf.coef_
        L1norm10k[count,i] = np.sum(np.abs(X[i,:]))
        L2norm10k[count,i] = np.sum((np.dot(A,X[i,:])[np.newaxis, :].T-ytemp)**2)
        y10kStored = ytemp
    LX10k[count,:,:]=X
    count=count+1

In [ ]:
# write data with cPickle
cPickle.dump( LX10k, open( "Lasso10kSolutionstest.pkl", "wb" ) )

In [ ]:
# 10k Solution Averages
L1norm10kMean = np.mean(L1norm10k,axis=1)
L2norm10kMean = np.mean(L2norm10k,axis=1)
L1norm10kStd = np.std(L1norm10k,axis=1)
L2norm10kStd = np.std(L2norm10k,axis=1)

XlamMean10k = np.zeros((50,88))
XlamStd10k = np.zeros((50,88))
XlamMedian10k = np.zeros((50,88))
for lam in range(0,50):
    XlamMean10k[lam,:]=np.mean(LX10k[lam],axis=0)
    XlamMedian10k[lam,:]=np.median(LX10k[lam],axis=0)
    XlamStd10k[lam,:]=np.std(LX10k[lam],axis=0)
    

In [ ]:
#PLOTTING - Chi2 error
lam=20
plt.plot(As,L2norm10kMean)
plt.plot(As[lam],L2norm10kMean[lam],'rv',ms=10,fillstyle='full')
plt.xlabel('Sparsity parameter')
plt.ylabel('Chi2 error')
plt.xscale('log')
plt.show()

In [ ]:
#PLOTTING - Neuron weights
for i in range(0,88):
        plt.plot(As,XlamMedian10k[:,i])
        plt.xscale('log')
        plt.xlabel('Sparsity parameter')
        plt.ylabel('Neuron weights')
        if abs(XlamMedian10k[lam,i])>2:
            if i==10:
                plt.text(As[lam+2],XlamMedian10k[lam,i]+2,Neurons[i])
            else:
                plt.text(As[lam],XlamMedian10k[lam,i],Neurons[i])
plt.show()

In [ ]:
#PLOTTING - 10k Solutions box plot
data=LX10k[lam]
box = plt.boxplot(data, notch=False, patch_artist=False, whis=[5,95])
plt.ylim([-20, 80])
plt.xlabel('Neurons')
plt.ylabel('Neuron weights')

for i in range(0,88):
    if abs(XlamMedian10k[lam,i])>2:
        plt.text(i+2,XlamMedian10k[lam,i],Neurons[i])
plt.show()

#### Binary Phenotype Vector

In [ ]:
# Binary phenotype vector
ybinary=np.array([ 1.0,  0. ,  0. ,  0. ,  0. ,  1.0 ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1.0 ,  0. ,  1.0 ,  0. ,  0. ,  0. ,  1.0])

In [ ]:
# Lasso solution w/ binary ophenotype vector
XBinary=np.zeros((50,88))
Asb=np.logspace(-4,0,num=50) #Sparsity Parameters
L1normBinary=np.zeros((50,1))
L2normBinary=np.zeros((50,1))
Score=np.zeros((50,1))
count=0
for a in Asb:
    clf = Lasso(alpha=a, max_iter=5000,)
    clf.fit(A, ybinary)
    Score[count]=clf.score(A, ybinary)
    XBinary[count,:] = clf.coef_
    L1normBinary[count] = np.sum(np.abs(XBinary[count,:]))
    L2normBinary[count] = np.sum((np.dot(A,XBinary[count,:])-ybinary)**2)
    count=count+1

In [ ]:
# PLOTTING - Binary phenotype vector solution
for i in range(0,88):
    plt.plot(Asb,XBinary[:,i])
    plt.xlabel('Sparsity parameter')
    plt.ylabel('Neuron weights')
    if abs(XlamMedian10k[lam,i])>2:
        if i==10:
            plt.text(Asb[lam]-0.002,XBinary[lam,i]-0.04,Neurons[i])
        else:
            plt.text(Asb[lam],XBinary[lam,i],Neurons[i])
plt.xscale('log')
plt.show()

In [ ]:
lam=20
plt.plot(Asb,L2normBinary)
plt.plot(Asb[lam],L2normBinary[lam],'rv',ms=10,fillstyle='full')
plt.xlabel('Sparsity parameter')
plt.ylabel('Chi2 error')
plt.xscale('log')
plt.show()

In [ ]:
plt.bar(np.arange(1,89), XBinary[lam,:])
plt.xlabel('Neurons')
plt.ylabel('Neuron weights')
for i in range(0,88):
    if abs(XlamMedian10k[lam,i])>2:
        plt.text(i+1,XBinary[lam,i],Neurons[i])
plt.show()

#### saving workspace - optional

In [ ]:
import dill

In [ ]:
# Save workspace
filename = 'workspace.pkl'
dill.dump_session(filename)

In [ ]:
# load workspace
filename = 'workspace.pkl'
dill.load_session(filename)